In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Загрузка данных MNIST
full_train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
full_test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 90853431.27it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 78761829.53it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 66468278.07it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 18406307.99it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
import random
import numpy as np

# Разбиение обучающих и тестовых данных

# Используем все тестовые данные для проверки
test_dataset = full_test_dataset

# Разбиение обучающих данных

# Количество меток (от 0 до 9)
labels_num = int(10)
# Длина всего обучающего множества и того обучающего множества, которое мы хотим использовать для обучения
full_train_len = len(full_train_dataset)
# Длина того обучающего множества, которое мы хотим использовать для обучения, оно должно делиться на количество меток
train_len = int(100)
# Количество данных с одной меткой
label_group_num = int(train_len/labels_num)

# Создаём группы для хранения индексов каждой метки в обучающем наборе данных
label_groups_index = [[] for _ in range(labels_num)]
for i in range(full_train_len):
  label = full_train_dataset[i][1]
  label_groups_index[label].append(i)

# Обрезаем группы, оставляя случайные, неповторяющиеся элементы в каждой и объединяем их всех в один набор индексов
all_index = np.array([], dtype=int)
for i in range(labels_num):
  all_index = np.append(all_index, random.sample(label_groups_index[i], label_group_num))
np.random.shuffle(all_index)

# Формируем обучающий набор данных
train_dataset = torch.utils.data.Subset(full_train_dataset, all_index)

In [ ]:
# Параметры обучения
batch_size = 64
learning_rate = 0.1
epochs = 5

# Определение модели
# neurons_num - количество нейронов в каждом слое
# img_size - размер изображений из MNIST
class Net(nn.Module):
    def __init__(self, neurons_num, img_size):
        super(Net, self).__init__()
        self.img_size = img_size
        self.fc1 = nn.Linear(img_size, neurons_num)
        self.fc2 = nn.Linear(neurons_num, neurons_num)
        self.fc3 = nn.Linear(neurons_num, neurons_num)
        self.fc4 = nn.Linear(neurons_num, neurons_num)
        self.fc5 = nn.Linear(neurons_num, neurons_num)
        self.fc6 = nn.Linear(neurons_num, 10)

    def forward(self, x):
        x = x.view(-1, self.img_size)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.relu(self.fc3(x))
        x = nn.functional.relu(self.fc4(x))
        x = nn.functional.relu(self.fc5(x))
        x = self.fc6(x)
        return x


# Инициализация модели и оптимизатора
model = Net(1000, 28*28)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Инициализация загрузчиков данных
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

loss_array = []
accuracy_array = []

# Обучение модели
#for epoch in range(epochs):
epoch = 0
while True:
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = nn.functional.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

    # Проверка точности модели на тестовом наборе данных
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    loss_array.append(loss.item())
    accuracy_array.append(accuracy)
    print('Epoch [{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, loss.item(), accuracy))
    epoch+=1
    if accuracy >= 88.5:
      break

Epoch [1], Loss: 2.3021, Accuracy: 12.91%
Epoch [2], Loss: 2.3033, Accuracy: 10.10%
Epoch [3], Loss: 2.3090, Accuracy: 10.10%
Epoch [4], Loss: 2.3095, Accuracy: 10.12%
Epoch [5], Loss: 2.3000, Accuracy: 10.10%
Epoch [6], Loss: 2.2998, Accuracy: 10.10%
Epoch [7], Loss: 2.3073, Accuracy: 10.20%
Epoch [8], Loss: 2.3038, Accuracy: 10.23%
Epoch [9], Loss: 2.3011, Accuracy: 10.41%
Epoch [10], Loss: 2.3039, Accuracy: 12.43%
Epoch [11], Loss: 2.3024, Accuracy: 14.99%
Epoch [12], Loss: 2.3025, Accuracy: 17.79%
Epoch [13], Loss: 2.3003, Accuracy: 14.14%
Epoch [14], Loss: 2.3042, Accuracy: 13.05%
Epoch [15], Loss: 2.3022, Accuracy: 13.74%
Epoch [16], Loss: 2.3015, Accuracy: 16.43%
Epoch [17], Loss: 2.3005, Accuracy: 19.56%
Epoch [18], Loss: 2.3027, Accuracy: 15.48%
Epoch [19], Loss: 2.3045, Accuracy: 10.11%
Epoch [20], Loss: 2.2997, Accuracy: 15.90%
Epoch [21], Loss: 2.2977, Accuracy: 12.08%
Epoch [22], Loss: 2.2984, Accuracy: 10.10%
Epoch [23], Loss: 2.3000, Accuracy: 10.10%
Epoch [24], Loss: 2.

KeyboardInterrupt: ignored

In [ ]:
for epoch in range(epochs):
  print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, epochs, loss_array[epoch], accuracy_array[epoch]))

Epoch [1/10], Loss: 0.3109, Accuracy: 93.68%
Epoch [2/10], Loss: 0.0239, Accuracy: 96.76%
Epoch [3/10], Loss: 0.0804, Accuracy: 96.81%
Epoch [4/10], Loss: 0.0042, Accuracy: 97.54%
Epoch [5/10], Loss: 0.0829, Accuracy: 96.98%
Epoch [6/10], Loss: 0.1174, Accuracy: 94.79%
Epoch [7/10], Loss: 0.0011, Accuracy: 97.85%
Epoch [8/10], Loss: 0.0002, Accuracy: 98.23%
Epoch [9/10], Loss: 0.0200, Accuracy: 97.69%
Epoch [10/10], Loss: 0.0001, Accuracy: 98.15%
